# Amazon Bedrock API


In [ ]:
# Test the Bedrock setup

import boto3
bedrock = boto3.client(service_name='bedrock')

bedrock.get_foundation_model(modelIdentifier='anthropic.claude-v2')

In [ ]:
# Test Bedrock Runtime

import boto3
import json

bedrock = boto3.client(service_name='bedrock-runtime')

body = json.dumps({
    "prompt": "\n\nHuman:explain black holes to 8th graders?\n\nAssistant:",
    "max_tokens_to_sample": 1024,
    "temperature": 0.1,
    "top_p": 0.9,
})

modelId = 'anthropic.claude-v2'

accept = 'application/json'

contentType = 'application/json'

response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)

response_body = json.loads(response.get('body').read())

# text
print(response_body.get('completion'))


In [ ]:
# Invoke the model with streaming code

import boto3
import json

brt = boto3.client(service_name='bedrock-runtime')

body = json.dumps({
    'prompt': '\n\nHuman: write an essay for living on mars in 1000 words\n\nAssistant:',
    'max_tokens_to_sample': 1024
})
                   
response = brt.invoke_model_with_response_stream(
    modelId='anthropic.claude-v2', 
    body=body
)
    
stream = response.get('body')
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            print(json.loads(chunk.get('bytes').decode()))

In [ ]:
import boto3 
bedrock = boto3.client(service_name='bedrock')

bedrock.list_foundation_models()

In [ ]:
import boto3 
bedrock = boto3.client(service_name='bedrock')

bedrock.get_foundation_model(modelIdentifier='anthropic.claude-v2')

Two types of Orchestration of Bedrock.

With Amazon Bedrock, you can enable a Retrieval-Augmented Generation (RAG)ß workflow by using knowledge bases to build contextual applications by using the reasoning capabilities of LLMs. To use a knowledge base, choose Orchestration in the left navigation pane and then Knowledge base. For more information, see Knowledge base for Amazon Bedrock.

Agents for Amazon Bedrock enables developers to configure an agent to complete actions based on organization data and user input. For example you might create an agent to take actions to fulfill a customer's request. To use an Agent, choose Orchestration in the left navigation pane and then Agent. For more information, see Agents for Amazon Bedrock.

In [ ]:
# Try with Claude3 Sonnet

In [1]:
import boto3
import json
import logging

from botocore.exceptions import ClientError

In [2]:
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)

In [2]:
def generate_message(bedrock_runtime, model_id, system_prompt, messages, max_tokens):
    
    body = json.dumps(
    
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "system": system_prompt,
            "messages": messages
        }
    )
    
    response = bedrock_runtime.invoke_model(body=body, modelId=model_id)
    response_body = json.loads(response.get('body').read())
    
    return response_body

In [3]:
# can put these into a main function

bedrock_runtime = boto3.client(service_name='bedrock-runtime')

model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'

system_prompt = "Please response with user's input language, and be an expert as you can."

max_tokens = 4096

In [8]:
user_message = {"role": "user", "content": "Can you recognize PDF file?"}

messages = [user_message]

response = generate_message(bedrock_runtime, model_id, system_prompt, messages, max_tokens)

print("User turn only.")
print(json.dumps(response, indent=4))

User turn only.
{
    "id": "msg_01Tw8nnqqY49rmJGnxdwSuEW",
    "type": "message",
    "role": "assistant",
    "content": [
        {
            "type": "text",
            "text": "Yes, I have the capability to recognize and analyze PDF (Portable Document Format) files. PDF is a widely used file format for sharing and distributing documents in a format that preserves the original formatting and layout. Please provide me with the PDF file you would like me to analyze, and I will do my best to extract and summarize the relevant information from it."
        }
    ],
    "model": "claude-3-sonnet-28k-20240229",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
        "input_tokens": 29,
        "output_tokens": 78
    }
}


In [9]:
assistant_message = {"role": "assistant", "content": "<emoji>"}
messages = [user_message, assistant_message]
response = generate_message(bedrock_runtime, model_id, system_prompt, messages, max_tokens)
print("User turn and prefilled assistant response.")
print(json.dumps(response, indent=4))

User turn and prefilled assistant response.
{
    "id": "msg_01V9c3r6A3h9nwtHUK16p1QS",
    "type": "message",
    "role": "assistant",
    "content": [
        {
            "type": "text",
            "text": "\ud83d\udc4d</emoji> Yes, I am able to recognize and understand PDF (Portable Document Format) files. PDF is a widely-used file format for representing documents in a way that preserves formatting and layout across different devices and operating systems. I have the capability to extract text, images, and other content from PDF files for analysis or processing. Please feel free to provide me with a PDF file, and I will do my best to interpret and work with its content based on your requirements."
        }
    ],
    "model": "claude-3-sonnet-28k-20240229",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
        "input_tokens": 32,
        "output_tokens": 103
    }
}


In [11]:
# Run a multi-modal claude3 sonnet example.

In [13]:
import json
import logging
import base64
import boto3

from botocore.exceptions import ClientError

In [14]:
#logger = logging.getLogger(__name__)
#logging.basicConfig(level=logging.INFO)

In [15]:
def run_multi_modal_prompt(bedrock_runtime, model_id, messages, max_tokens):
    """
    Inovke a model with a multimodal prompt.
    
    Args:
        bedrock_runtime: The amazon Bedrock boto3 client.
        model_id (str): The model ID to use.
        messages (JSON): The messages to send to the model.
        max_tokens (int): The maximum number of tokens to generate.
        
    Returns:
        None.
    """
    
    body = json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "messages": messages
        }
    )
    
    response = bedrock_runtime.invoke_model(body=body, modelId=model_id)
    response_body = json.loads(response.get('body').read())
    
    return response_body

In [16]:
# The following can be put into a main function.

bedrock_runtime = boto3.client(service_name='bedrock-runtime')

model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'

max_tokens = 4096

input_image = "./img/catdog.jpg"

input_text = "What is in this image?"

In [17]:
with open(input_image, "rb") as image_file:
    content_image = base64.b64encode(image_file.read()).decode('utf8')

In [21]:
message = {
    
    "role": "user",
    "content": [
        {
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/jpeg",
                "data": content_image
            }
        },
        {
            "type": "text",
            "text": input_text
        }
    ]
}

messages = [message]

In [22]:
response = run_multi_modal_prompt(bedrock_runtime, model_id, messages, max_tokens)
print(json.dumps(response, indent=4))

{
    "id": "msg_01FauRhDTSyATFBxpt1umypi",
    "type": "message",
    "role": "assistant",
    "content": [
        {
            "type": "text",
            "text": "This image depicts a dog and a cat sitting next to each other. The dog appears to be a tan-colored pug with large, expressive eyes. The cat is gray striped and has its mouth open, seeming to meow or make a vocalization. Both animals are positioned facing the camera against a plain background. The contrasting expressions and poses of the pug and cat create an amusing, light-hearted scene capturing the interactions between the two different pet species."
        }
    ],
    "model": "claude-3-sonnet-28k-20240229",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
        "input_tokens": 592,
        "output_tokens": 101
    }
}
